In [17]:
import json
import numpy as np


def qmult(q1, q0):
    x0, y0, z0, w0 = q0
    x1, y1, z1, w1 = q1
    return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)

file = "testaprilpose.json"
with open(file, 'r') as f1:
    testaprilpose_data = json.load(f1)
type(testaprilpose_data)
tag_read_loc=testaprilpose_data["TAG_0"]

tag_read = [-.5032, -.2147,1.3717,.9983,.0003,.0012,.0575]
tag_read_map_coord_XYZ = [tag_read[0],-tag_read[2],tag_read[1]]
map_coord_trans = [0.0,0.0,0.0,0.0]
tag_read_map_coord_Q = qmult(tag_read[3:],tag_read_loc[3:])




array([ 0.04150677,  0.7056858 ,  0.70611006, -0.03980973])